In [39]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import os

pfad_zu_dateien = "/Users/zen/Documents/Data and information/Semester 3/Retrieval/" 


dateien = [
    "Chemnitz_blaulicht_scrape_2025-12-07.csv",
    "Dortmund_blaulicht_scrape_2025-12-07.csv",
    "Saarbruecken_blaulicht_scrape_2025-12-09-2.csv",
    "Nuernberg_blaulicht_scrape_2025-12-09.csv",
    "Erfurt_blaulicht_scrape_2025-12-09.csv"
]

all_docs = []
doc_metadata = []

print("Lade Dateien und bereite Daten vor...")

for dateiname in dateien:
    voller_pfad = os.path.join(pfad_zu_dateien, dateiname)
    
    if os.path.exists(voller_pfad):
        try:
            # Daten laden
            df = pd.read_csv(voller_pfad, on_bad_lines='skip')
            df.columns = [c.lower() for c in df.columns] # Alles kleinschreiben
            
            # Text kombinieren
            if 'title' in df.columns and 'abstract' in df.columns:
                df['text'] = df['title'].astype(str) + " " + df['abstract'].astype(str)
            else:
                df['text'] = df.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
            
            # Stadt aus Dateinamen extrahieren (alles vor dem ersten Unterstrich)
            stadt_name = dateiname.split('_')[0]
            
            # Speichern für TF-IDF
            for idx, text in enumerate(df['text']):
                all_docs.append(text)
                doc_metadata.append({'Stadt': stadt_name, 'Original_Datei': dateiname})
                
        except Exception as e:
            print(f"Fehler beim Lesen von {dateiname}: {e}")
    else:
        print(f"DATEI NICHT GEFUNDEN: {voller_pfad} - Pfad prüfen!")

# Wenn keine Dokumente geladen wurden, abbrechen
if not all_docs:
    print("Keine Daten geladen. Bitte Dateipfade überprüfen.")
else:
    print(f"{len(all_docs)} Dokumente geladen. Berechne TF-IDF...")

    # TF-IDF Berechnung
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(all_docs)
    feature_names = vectorizer.get_feature_names_out()

    # Indizes für Keywords finden
    suchbegriffe = ['drogen', 'kokain', 'koks', 'droge', 'mdma', 'methamphetamin', 'methamphetamine']
    target_indices = [np.where(feature_names == k)[0][0] for k in suchbegriffe if k in feature_names]

    if target_indices:
        
        scores = np.asarray(tfidf_matrix[:, target_indices].sum(axis=1)).flatten()
        
        # Ergebnisse zusammenstellen
        ergebnis_liste = []
        for i, meta in enumerate(doc_metadata):
            ergebnis_liste.append({
                'Stadt': meta['Stadt'],
                'Score': scores[i]
            })
        
        ergebnis_df = pd.DataFrame(ergebnis_liste)
        
        # AGGREGATION: Zusammenfassung pro Stadt erstellen
        zusammenfassung = ergebnis_df.groupby('Stadt').agg(
            Gesamt_Berichte=('Score', 'count'),
            Relevante_Berichte=('Score', lambda x: (x > 0).sum()), # Zählt Scores größer 0
            Durchschnitts_Score=('Score', 'mean')
        )
        
        # Frequenz in Prozent berechnen
        zusammenfassung['Frequenz_Prozent'] = (zusammenfassung['Relevante_Berichte'] / zusammenfassung['Gesamt_Berichte']) * 100
        
        # Sortieren nach Frequenz (Beste zuerst)
        zusammenfassung = zusammenfassung.sort_values(by='Frequenz_Prozent', ascending=False)
        
        print("\n--- Relevanz-Vergleich der Städte ---")
        print(zusammenfassung.round(4)) 
        
    else:
        print("Die Suchbegriffe 'Drogen' oder 'Kokain' nicht gefunden.")

Lade Dateien und bereite Daten vor...
21223 Dokumente geladen. Berechne TF-IDF...

--- Relevanz-Vergleich der Städte ---
              Gesamt_Berichte  Relevante_Berichte  Durchschnitts_Score  \
Stadt                                                                    
Dortmund                10800                 355               0.0044   
Chemnitz                  687                  12               0.0020   
Erfurt                   5000                  80               0.0023   
Saarbruecken             1871                  27               0.0014   
Nuernberg                2865                  37               0.0018   

              Frequenz_Prozent  
Stadt                           
Dortmund                3.2870  
Chemnitz                1.7467  
Erfurt                  1.6000  
Saarbruecken            1.4431  
Nuernberg               1.2914  


In [44]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import os
import re # Wichtig für das Auslesen der Jahreszahl

# Ihr Pfad
pfad_zu_dateien = "/Users/zen/Documents/Data and information/Semester 3/Retrieval/" 

dateien = [
    "Chemnitz_blaulicht_scrape_2025-12-07.csv",
    "Dortmund_blaulicht_scrape_2025-12-07.csv",
    "Saarbruecken_blaulicht_scrape_2025-12-09-2.csv",
    "Nuernberg_blaulicht_scrape_2025-12-09.csv",
    "Erfurt_blaulicht_scrape_2025-12-09.csv",
    "München_blaulicht_scrape_2025-12-18.csv"
]

all_docs = []
doc_metadata = []

print("Lade Dateien und bereite Daten vor...")

for dateiname in dateien:
    voller_pfad = os.path.join(pfad_zu_dateien, dateiname)
    
    # Prüfen, ob die Datei existiert
    if os.path.exists(voller_pfad):
        try:
            # Daten laden
            df = pd.read_csv(voller_pfad, on_bad_lines='skip')
            df.columns = [c.lower() for c in df.columns] # Alles kleinschreiben
            
            # 1. Text kombinieren
            if 'title' in df.columns and 'abstract' in df.columns:
                df['text'] = df['title'].astype(str) + " " + df['abstract'].astype(str)
            else:
                df['text'] = df.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
            
            # 2. Stadt aus Dateinamen extrahieren
            stadt_name = dateiname.split('_')[0]
            
            # 3. Jahr aus Datum extrahieren (NEU)
            # Wir suchen nach 4 aufeinanderfolgenden Ziffern (z.B. "2023") in der Spalte 'datum'
            if 'datum' in df.columns:
                df['jahr'] = df['datum'].astype(str).str.extract(r'(\d{4})')
            else:
                df['jahr'] = 'Unbekannt'
            
            # Speichern für TF-IDF (Text + Metadaten pro Zeile)
            # Wir nutzen zip(), um Text und Jahr gleichzeitig durchzugehen
            for text, jahr in zip(df['text'], df['jahr']):
                all_docs.append(text)
                doc_metadata.append({
                    'Stadt': stadt_name, 
                    'Jahr': jahr,
                    'Original_Datei': dateiname
                })
                
        except Exception as e:
            print(f"Fehler beim Lesen von {dateiname}: {e}")
    else:
        print(f"DATEI NICHT GEFUNDEN: {voller_pfad} - Pfad prüfen!")

# Wenn keine Dokumente geladen wurden, abbrechen
if not all_docs:
    print("Keine Daten geladen. Bitte Dateipfade überprüfen.")
else:
    print(f"{len(all_docs)} Dokumente geladen. Berechne TF-IDF...")

    # TF-IDF Berechnung
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(all_docs)
    feature_names = vectorizer.get_feature_names_out()

    # Ihre erweiterte Liste
    suchbegriffe = ['drogen', 'kokain', 'koks', 'droge', 'mdma', 'methamphetamin', 'methamphetamine']
    
    # Indizes finden
    target_indices = [np.where(feature_names == k)[0][0] for k in suchbegriffe if k in feature_names]

    if target_indices:
        # Summe der Scores für die Suchbegriffe
        scores = np.asarray(tfidf_matrix[:, target_indices].sum(axis=1)).flatten()
        
        # Ergebnisse zusammenstellen
        ergebnis_liste = []
        for i, meta in enumerate(doc_metadata):
            ergebnis_liste.append({
                'Stadt': meta['Stadt'],
                'Jahr': meta['Jahr'], # Das Jahr muss hier mit rein
                'Score': scores[i]
            })
        
        ergebnis_df = pd.DataFrame(ergebnis_liste)
        
        # Leere Jahre (NaN) entfernen, falls vorhanden
        ergebnis_df = ergebnis_df.dropna(subset=['Jahr'])
        
        # AGGREGATION: Jetzt gruppieren wir nach Stadt UND Jahr
        zusammenfassung = ergebnis_df.groupby(['Stadt', 'Jahr']).agg(
            Gesamt_Berichte=('Score', 'count'),
            Relevante_Berichte=('Score', lambda x: (x > 0).sum()), 
            Durchschnitts_Score=('Score', 'mean')
        ).reset_index() # reset_index macht aus dem Gruppen-Index wieder normale Spalten
        
        # Frequenz in Prozent berechnen
        zusammenfassung['Frequenz_Prozent'] = (zusammenfassung['Relevante_Berichte'] / zusammenfassung['Gesamt_Berichte']) * 100
        
        zusammenfassung = zusammenfassung.round(4)
        
        # Sortieren: Zuerst nach Stadt, dann nach Jahr (chronologisch)
        zusammenfassung = zusammenfassung.sort_values(by=['Stadt', 'Jahr'])
        
        print("\n--- Relevanz-Vergleich der Städte nach Jahren ---")
        # to_string() sorgt dafür, dass alle Zeilen angezeigt werden
        print(zusammenfassung.to_string()) 
        
    else:
        print("Keine der Suchbegriffe im Textkorpus gefunden.")

    zusammenfassung.to_csv('drogen_analyse_jahresvergleich.csv', index=False, sep=';', decimal=',')

print("Datei wurde erfolgreich gespeichert!")

Lade Dateien und bereite Daten vor...
28248 Dokumente geladen. Berechne TF-IDF...

--- Relevanz-Vergleich der Städte nach Jahren ---
           Stadt  Jahr  Gesamt_Berichte  Relevante_Berichte  Durchschnitts_Score  Frequenz_Prozent
0       Chemnitz  2018                1                   0               0.0000            0.0000
1       Chemnitz  2019               15                   0               0.0000            0.0000
2       Chemnitz  2020               56                   2               0.0046            3.5714
3       Chemnitz  2021               62                   0               0.0000            0.0000
4       Chemnitz  2022               90                   0               0.0000            0.0000
5       Chemnitz  2023              141                   3               0.0024            2.1277
6       Chemnitz  2024               80                   3               0.0038            3.7500
7       Chemnitz  2025              242                   4               0

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import os
import re

# --- KONFIGURATION: DATENQUELLEN (GITHUB) ---
base_url = "https://raw.githubusercontent.com/ichzann/DIS08_Data_modeling_7er_Gruppe/main/Daten_sets/blaulicht_scraping/"

# Liste der Dateien im Repo 
folders_csv_in_repo = [
    "Chemnitz_blaulicht_scrape_2025-12-07.csv",
    "Dortmund_blaulicht_scrape_2025-12-07.csv",
    "Erfurt_blaulicht_scrape_2025-12-09.csv",
    "Nuernberg_blaulicht_scrape_2025-12-09.csv",
    "Saarbruecken_blaulicht_scrape_2025-12-09.csv",
    "München_blaulicht_scrape_2025-12-18.csv"
]

all_docs = []
doc_metadata = []

print("Lade Daten von GitHub und bereite vor...")

# --- SCHRITT 1: DATEN LADEN ---
for file_name in folders_csv_in_repo:
    # URL zusammenbauen
    url = f"{base_url}{file_name}"
    
    try:
        # Pandas kann CSVs direkt von URLs lesen
        # storage_options={'User-Agent': 'Mozilla/5.0'} hilft manchmal bei Zugriffsproblemen, meist geht es aber ohne
        df = pd.read_csv(url, on_bad_lines='skip')
        
        # Spalten bereinigen
        df.columns = [c.lower() for c in df.columns]
        
        # 1. Text kombinieren (Title + Abstract)
        # Wir prüfen sicherheitshalber, welche Spalten da sind
        if 'title' in df.columns and 'abstract' in df.columns:
            df['text'] = df['title'].astype(str) + " " + df['abstract'].astype(str)
        else:
            # Fallback: Alles verbinden, falls Spaltennamen anders sind
            df['text'] = df.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
        
        # 2. Stadt aus Dateinamen extrahieren
        # Split am Unterstrich (z.B. "Chemnitz_...")
        stadt_name = file_name.split('_')[0]
        
        # 3. Jahr aus Datum extrahieren
        # Wir nutzen hier die robuste Regex-Methode, da sie Format-Unabhängig ist
        if 'datum' in df.columns:
            df['jahr'] = df['datum'].astype(str).str.extract(r'(\d{4})')
        else:
            df['jahr'] = 'Unbekannt'
        
        # Daten sammeln für die Analyse
        # Wir iterieren durch den DataFrame und speichern Text + Metadaten
        for text, jahr in zip(df['text'], df['jahr']):
            all_docs.append(text)
            doc_metadata.append({
                'Stadt': stadt_name, 
                'Jahr': jahr,
                'Original_Datei': file_name
            })
            
        print(f"OK: {file_name} geladen.")

    except Exception as e:
        print(f"FEHLER bei {file_name}: {e}")

# --- SCHRITT 2: ANALYSE (TF-IDF) ---
if not all_docs:
    print("Keine Datengeladen.")
else:
    print(f"\n{len(all_docs)} Dokumente insgesamt. TF-IDF Analyse")

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(all_docs)
    feature_names = vectorizer.get_feature_names_out()

    #Keyword-Liste
    suchbegriffe = ['drogen', 'kokain', 'koks', 'droge', 'mdma', 'methamphetamin', 'methamphetamine']
    
    # Indizes der Keywords im Vokabular finden
    target_indices = [np.where(feature_names == k)[0][0] for k in suchbegriffe if k in feature_names]

    if target_indices:
        # Summe der Scores berechnen
        scores = np.asarray(tfidf_matrix[:, target_indices].sum(axis=1)).flatten()
        
        # Ergebnisse verknüpfen
        ergebnis_liste = []
        for i, meta in enumerate(doc_metadata):
            ergebnis_liste.append({
                'Stadt': meta['Stadt'],
                'Jahr': meta['Jahr'],
                'Score': scores[i]
            })
        
        ergebnis_df = pd.DataFrame(ergebnis_liste)
        
        # Ungültige Jahre entfernen
        ergebnis_df = ergebnis_df.dropna(subset=['Jahr'])
        
        # --- SCHRITT 3: AGGREGATION & DIFFERENZEN ---
        
        # Gruppieren nach Stadt und Jahr
        zusammenfassung = ergebnis_df.groupby(['Stadt', 'Jahr']).agg(
            Gesamt_Berichte=('Score', 'count'),
            Relevante_Berichte=('Score', lambda x: (x > 0).sum()), 
            Durchschnitts_Score=('Score', 'mean')                       ####Raus score nicht nötig 
        ).reset_index()
        
        # Frequenz berechnen
        zusammenfassung['Frequenz_Prozent'] = (zusammenfassung['Relevante_Berichte'] / zusammenfassung['Gesamt_Berichte']) * 100
        
        #Sortieren (Stadt -> Jahr), damit die Differenzberechnung stimmt
        zusammenfassung = zusammenfassung.sort_values(by=['Stadt', 'Jahr'])

        # Differenzen berechnen Veränderung zum Vorjahr innerhalb der gleichen Stadt
        zusammenfassung['Diff_Score'] = zusammenfassung.groupby('Stadt')['Durchschnitts_Score'].diff()      ####Raus score nicht nötig 
        zusammenfassung['Diff_Frequenz'] = zusammenfassung.groupby('Stadt')['Frequenz_Prozent'].diff()

        # Runden
        zusammenfassung = zusammenfassung.round(4)
        
        # Ausgabe im Terminal
        print("\n--- Analyse-Ergebnis ---")
        print(zusammenfassung.to_string()) 
        
        # Speichern
        output_filename = 'drogen_analyse_github_daten.csv'
        zusammenfassung.to_csv(output_filename, index=False, sep=';', decimal=',')
        print(f"\nDatei erfolgreich gespeichert: {output_filename}")

    else:
        print("Keine der Suchbegriffe im Textkorpus gefunden.")

Lade Daten von GitHub und bereite vor...
OK: Chemnitz_blaulicht_scrape_2025-12-07.csv geladen.
OK: Dortmund_blaulicht_scrape_2025-12-07.csv geladen.
OK: Erfurt_blaulicht_scrape_2025-12-09.csv geladen.
OK: Nuernberg_blaulicht_scrape_2025-12-09.csv geladen.
OK: Saarbruecken_blaulicht_scrape_2025-12-09.csv geladen.
FEHLER bei München_blaulicht_scrape_2025-12-18.csv: 'ascii' codec can't encode character '\u0308' in position 81: ordinal not in range(128)

21223 Dokumente insgesamt. Starte TF-IDF Analyse...

--- Analyse-Ergebnis (Auszug) ---
           Stadt  Jahr  Gesamt_Berichte  Relevante_Berichte  Durchschnitts_Score  Frequenz_Prozent  Diff_Score  Diff_Frequenz
0       Chemnitz  2018                1                   0               0.0000            0.0000         NaN            NaN
1       Chemnitz  2019               15                   0               0.0000            0.0000      0.0000         0.0000
2       Chemnitz  2020               56                   2               0.004